In [ ]:
!pip install itables

import itables.options as opt

from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=True)

In [ ]:
from functools import partial

from rank_yogo_loss import get_loss_df, show_image

In [ ]:
PATH_TO_MODEL = (
    "/hpc/projects/flexo/MicroscopyData/Bioengineering"
    "/LFM_scope/yogo_models/honest-sweep-51/best.pth"
)

# I think I may need to put this dataset defn. file somewhere else
PATH_TO_ALL_LABELLED_DATA_DDF = (
    "/home/axel.jacobsen/celldiagnosis/dataset_defs/all-labelled-data-train-only.yml"
)

In [ ]:
df = get_loss_df(PATH_TO_ALL_LABELLED_DATA_DDF, PATH_TO_MODEL)


In [ ]:
show_image = partial(show_image, df=df, path_to_pth=PATH_TO_MODEL)

opt.maxRows = len(df)
opt.maxBytes = 250000000

In [ ]:
show(df)

In [ ]:
show_image(df['total_loss'].idxmax())